# SPAM detection

For this code along we will build a spam filter!

We'll use a classic dataset for this - UCI Repository SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

#### load and  read the dataset,  have Spark infer the data types

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('NLP_PROJET').getOrCreate()

In [3]:
# Seperating the columns based on "tab"
data = spark.read.option("delimiter", '\t').csv('SMSSpamCollection')

In [4]:
# Renaming the default column names to the name mentioned in the requested result
data=data.withColumnRenamed("_c0", "class").withColumnRenamed("_c1", "text")

In [5]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



## Clean and Prepare the Data

#### Create a new length feature

In [6]:
#to get the length feature and compute average in later steps
from pyspark.sql.functions import length, avg 

data = data.withColumn('length', length(data.text))

In [7]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



#### print the groupy mean of class

In [8]:
data.groupBy("class").avg("length").show() #Calculate average of the feature "length"

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



## Feature Transformations

In this part you transform you raw text in to tf_idf model :

- chain the transformer Tokenizer, StopWordsRemover, CountVectorizer and IDF for text to have a final column name 'tf_idf'
- use the transformer StringIndexer for class column into output column 'label'

- create feature with vector assembler 'tf_idf','length of as input columns into output column named 'features'

In [9]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline

In [10]:
tokenizer = Tokenizer(inputCol="text", outputCol="words")


In [11]:
# Setting the system language to "en-US", since my laptop language is "en-FR"
locale = spark._jvm.java.util.Locale
locale.setDefault(locale.forLanguageTag("en-US"))
# StopWordsRemover will be making use of the system's language dictionary to work

In [12]:
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")


In [13]:
cv = CountVectorizer(inputCol="filtered_words", outputCol="rawFeatures")


In [14]:
idf = IDF(inputCol="rawFeatures", outputCol="tf_idf")


In [15]:
indexer = StringIndexer(inputCol="class", outputCol="label")

In [16]:
assembler = VectorAssembler(
    inputCols=["tf_idf", "length"],
    outputCol="features")

### use pipeline for fit and transform

In [17]:
pipeline = Pipeline(stages=[tokenizer,remover, cv, idf,indexer, assembler])

In [18]:
model = pipeline.fit(data)

In [19]:
result_data=model.transform(data)

In [20]:
clean_data=result_data.select('label','features')

In [21]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



### Detect spam or Ham

now use your tf-idf data to classify spam and ham

feel free to use any classifier model

result may differ for you

In [22]:
# Making use of the Naive Bayes Classification algorithm to predict
from pyspark.ml.classification import NaiveBayes


In [23]:
train_data, test_data = clean_data.randomSplit([0.7,.3])

In [24]:
features=train_data.select('features')
label=train_data.select('label')

In [25]:
nb = NaiveBayes(smoothing=1)
model = nb.fit(train_data)
test_results = model.transform(test_data)

In [26]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,7,8...|[-808.16379731185...|[1.0,2.4781460200...|       0.0|
|  0.0|(13424,[0,1,2,41,...|[-1067.0335237675...|[1.0,9.3727213646...|       0.0|
|  0.0|(13424,[0,1,5,15,...|[-998.02958364429...|[1.0,5.5351363915...|       0.0|
|  0.0|(13424,[0,1,5,20,...|[-794.34678720140...|[1.0,2.0213881860...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-1193.0812569252...|[1.0,2.2457531197...|       0.0|
|  0.0|(13424,[0,1,7,15,...|[-662.03025828160...|[1.0,4.4948174049...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-539.53131802429...|[1.0,3.0592221643...|       0.0|
|  0.0|(13424,[0,1,14,78...|[-704.49562682761...|[1.0,2.7264524959...|       0.0|
|  0.0|(13424,[0,1,21,27...|[-752.22193881088...|[1.0,1.5480479740...|       0.0|
|  0.0|(13424,[0

### Calculate the accuracy of your model

In [27]:
# We are making use of Binary Classification Evaluator, it is the best fit for this model
# As our labels are based on 0 & 1 classifciation

from pyspark.ml.evaluation import BinaryClassificationEvaluator

my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='label')
AUC = my_eval.evaluate(test_results)

In [28]:
print(AUC)

0.9372200263504612


We have 93% of accuracy